<a href="https://colab.research.google.com/github/catundchat/GPT_Prompt_Generator/blob/main/gpt_prompt_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT PROMPT GENERATOR

To generate a prompt:
1. In the SECOND cell, add your OpenAI key.
2. In the FOURTH cell, if you have GPT-4 access, you're ready to move on. If not, change CANDIDATE_MODEL='gpt-4' to CANDIDATE_MODEL='gpt-3.5-turbo'
3. In the FOURTH cell, change temperature, max_tokens and number_of_prompts
4. In the last but one cell, fill in the description of your task, up to 15 test cases.
5. Now all you need is to run all the cells. Good luck and have fun.

# 生成式预训练模型提示词生成器
为了生成一个提示词：
1. 在第二个单元格中，添加OpenAI密钥
2. 在第四个单元格中，如果你有GPT-4 API的访问权限,那你已经做好准备开展下一步了；如果你没有，则将"CANDIDATE_MODEL='gpt-4"改成"CANDIDATE_MODEL='gpt-3.5-turbo'"
3. 在第四个单元格中，更改如下参数：温度，最大令牌数和提示词数量
4. 在倒数第二个单元格中，修改你工作的描述，并补充最多15个测试用例
5. 现在可以执行所有单元格了。祝你好运

In [1]:
!pip install openai prettytable tqdm tenacity wandb -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.2/213.2 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [2]:
from prettytable import PrettyTable
import time
import openai
from tqdm import tqdm
import itertools
import wandb
from tenacity import retry, stop_after_attempt, wait_exponential

openai.api_key = "" # enter your OpenAI API key here

use_wandb = False # Weights & Biases (wandb) is used to track the procedure of generating prompts. if u wanna keep your data private, keep it False.

In [3]:
system_gen_system_prompt = """Your job is to generate system prompts for GPT-4, given a description of the use-case and some test cases.

The prompts you will be generating will be for freeform tasks, such as generating a landing page headline, an intro paragraph, solving a math problem, etc.

In your generated prompt, you should describe how the AI should behave in plain English. Include what it will see, and what it's allowed to output. Be creative with prompts to get the best possible results. The AI knows it's an AI -- you don't need to tell it this.

You will be graded based on the performance of your prompt... but don't cheat! You cannot include specifics about the test cases in your prompt. Any prompts with examples will be disqualified.

Most importantly, output NOTHING but the prompt. Do not include anything else in your message."""


ranking_system_prompt = """Your job is to rank the quality of two outputs generated by different prompts. The prompts are used to generate a response for a given task.

You will be provided with the task description, the test prompt, and two generations - one for each system prompt.

Rank the generations in order of quality. If Generation A is better, respond with 'A'. If Generation B is better, respond with 'B'.

Remember, to be considered 'better', a generation must not just be good, it must be noticeably superior to the other.

Also, keep in mind that you are a very harsh critic. Only rank a generation as better if it truly impresses you more than the other.

Respond with your ranking, and nothing else. Be fair and unbiased in your judgement."""

In [4]:
# K is a constant factor that determines how much ratings change
K = 32

CANDIDATE_MODEL = 'gpt-4' # CHANGE HERE IF U NEED
CANDIDATE_MODEL_TEMPERATURE = 0.9

GENERATION_MODEL = 'gpt-3.5-turbo'
GENERATION_MODEL_TEMPERATURE = 0.8
GENERATION_MODEL_MAX_TOKENS = 60

N_RETRIES = 3  # number of times to retry a call to the ranking model if it fails
RANKING_MODEL = 'gpt-3.5-turbo'
RANKING_MODEL_TEMPERATURE = 0.5

NUMBER_OF_PROMPTS = 7 # this determines how many candidate prompts to generate. The higher, the more expensive, but the better the results will be

WANDB_PROJECT_NAME = "LaunayViolette" # used if use_wandb is True, Weights &| Biases project name
WANDB_RUN_NAME = None # used if use_wandb is True, optionally set the Weights & Biases run name to identify this run

In [5]:
def start_wandb_run():
  # start a new wandb run and log the config
  wandb.init(
    project=WANDB_PROJECT_NAME,
    name=WANDB_RUN_NAME,
    config={
      "K": K,
      "system_gen_system_prompt": system_gen_system_prompt,
      "ranking_system_prompt": ranking_system_prompt,
      "candiate_model": CANDIDATE_MODEL,
      "candidate_model_temperature": CANDIDATE_MODEL_TEMPERATURE,
      "generation_model": GENERATION_MODEL,
      "generation_model_temperature": GENERATION_MODEL_TEMPERATURE,
      "generation_model_max_tokens": GENERATION_MODEL_MAX_TOKENS,
      "n_retries": N_RETRIES,
      "ranking_model": RANKING_MODEL,
      "ranking_model_temperature": RANKING_MODEL_TEMPERATURE,
      "number_of_prompts": NUMBER_OF_PROMPTS
      })

  return

In [6]:
# Optional logging to Weights & Biases to reocrd the configs, prompts and results
if use_wandb:
  start_wandb_run()

In [7]:
def generate_candidate_prompts(description, test_cases, number_of_prompts):
  outputs = openai.ChatCompletion.create(
      model=CANDIDATE_MODEL, # change this to gpt-3.5-turbo if you don't have GPT-4 access
      messages=[
          {"role": "system", "content": system_gen_system_prompt},
          {"role": "user", "content": f"Here are some test cases:`{test_cases}`\n\nHere is the description of the use-case: `{description.strip()}`\n\nRespond with your prompt, and nothing else. Be creative."}
          ],
      temperature=CANDIDATE_MODEL_TEMPERATURE,
      n=number_of_prompts)

  prompts = []

  for i in outputs.choices:
    prompts.append(i.message.content)
  return prompts

def expected_score(r1, r2):
    return 1 / (1 + 10**((r2 - r1) / 400))

def update_elo(r1, r2, score1):
    e1 = expected_score(r1, r2)
    e2 = expected_score(r2, r1)
    return r1 + K * (score1 - e1), r2 + K * ((1 - score1) - e2)

# Get Score - retry up to N_RETRIES times, waiting exponentially between retries.
@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def get_score(description, test_case, pos1, pos2, ranking_model_name, ranking_model_temperature):
    score = openai.ChatCompletion.create(
        model=ranking_model_name,
        messages=[
            {"role": "system", "content": ranking_system_prompt},
            {"role": "user", "content": f"""Task: {description.strip()}
Prompt: {test_case['prompt']}
Generation A: {pos1}
Generation B: {pos2}"""}
        ],
        logit_bias={
              '32': 100,  # 'A' token
              '33': 100,  # 'B' token
        },
        max_tokens=1,
        temperature=ranking_model_temperature,
    ).choices[0].message.content
    return score

@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def get_generation(prompt, test_case):
    generation = openai.ChatCompletion.create(
        model=GENERATION_MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"{test_case['prompt']}"}
        ],
        max_tokens=GENERATION_MODEL_MAX_TOKENS,
        temperature=GENERATION_MODEL_TEMPERATURE,
    ).choices[0].message.content
    return generation

def test_candidate_prompts(test_cases, description, prompts):
  # Initialize each prompt with an ELO rating of 1200
  prompt_ratings = {prompt: 1200 for prompt in prompts}

  # Calculate total rounds for progress bar
  total_rounds = len(test_cases) * len(prompts) * (len(prompts) - 1) // 2

  # Initialize progress bar
  pbar = tqdm(total=total_rounds, ncols=70)

  # For each pair of prompts
  for prompt1, prompt2 in itertools.combinations(prompts, 2):
      # For each test case
      for test_case in test_cases:
          # Update progress bar
          pbar.update()

          # Generate outputs for each prompt
          generation1 = get_generation(prompt1, test_case)
          generation2 = get_generation(prompt2, test_case)

          # Rank the outputs
          score1 = get_score(description, test_case, generation1, generation2, RANKING_MODEL, RANKING_MODEL_TEMPERATURE)
          score2 = get_score(description, test_case, generation2, generation1, RANKING_MODEL, RANKING_MODEL_TEMPERATURE)

          # Convert scores to numeric values
          score1 = 1 if score1 == 'A' else 0 if score1 == 'B' else 0.5
          score2 = 1 if score2 == 'B' else 0 if score2 == 'A' else 0.5

          # Average the scores
          score = (score1 + score2) / 2

          # Update ELO ratings
          r1, r2 = prompt_ratings[prompt1], prompt_ratings[prompt2]
          r1, r2 = update_elo(r1, r2, score)
          prompt_ratings[prompt1], prompt_ratings[prompt2] = r1, r2

          # Print the winner of this round
          if score > 0.5:
              print(f"Winner: {prompt1}")
          elif score < 0.5:
              print(f"Winner: {prompt2}")
          else:
              print("Draw")

  # Close progress bar
  pbar.close()

  return prompt_ratings


def generate_optimal_prompt(description, test_cases, number_of_prompts, use_wandb=False):
  if use_wandb:
    wandb_table = wandb.Table(columns=["Prompt", "Ranking"])
    if wandb.run is None:
      start_wandb_run()

  prompts = generate_candidate_prompts(description, test_cases, number_of_prompts)
  prompt_ratings = test_candidate_prompts(test_cases, description, prompts)

  # Print the final ELO ratingsz
  table = PrettyTable()
  table.field_names = ["Prompt", "Rating"]
  for prompt, rating in sorted(prompt_ratings.items(), key=lambda item: item[1], reverse=True):
      table.add_row([prompt, rating])
      if use_wandb:
         wandb_table.add_data(prompt, rating)

  if use_wandb: # log the results to a Weights & Biases table and finsih the run
    wandb.log({"prompt_ratings": wandb_table})
    wandb.finish()
  print(table)

In the cell below, fill in your description and test cases:

In [8]:
description = "这是一个心理学和亲密关系的对话模型，特别专注于育儿问题。模型应该能够理解和回应各种关于育儿的问题，提供有同理心和专业知识的答案。"

test_cases = [
    {
        "prompt": "我儿子总是不听话，我该怎么办？",
        "completion": "作为父母，我们理解你的困扰。首先，了解一下他的需求和感受是非常重要的。他可能是在寻求关注，或者在某些方面感到挫败。尝试和他进行沟通，了解他为什么不听话，然后提供适当的指导和支持。"
    },
    {
        "prompt": "我的女儿对学习没有兴趣，我应该怎么激励她？",
        "completion": "激发孩子对学习的兴趣可以是一项挑战。一个可能的方法是将学习与她的兴趣结合起来。如果她喜欢艺术，试试看能否找到一些结合艺术和学习的项目。另一个策略是设定合理的目标，并奖励她达成目标。"
    },
    {
        "prompt": "我感到育儿压力很大，我应该怎么缓解？",
        "completion": "育儿确实可以带来压力。记住，花时间照顾自己也是非常重要的。找一些你喜欢的事情去做，如阅读、运动或者和朋友聊天。如果压力过大，也可以寻求专业的心理咨询师帮助。"
    },
]

# description = "Given a prompt, generate a prompt to help large language model understand what user says and have better empathy and generate better responses"

# test_cases = [
#     {
#         'prompt': 'You are a very skilled psychology assistant with extensive knowledge of psychology and intimate relationship problem solving skills, and now you need to talk to users. Please empathise with your users and solve their problems as best as you can.'
#     }
# ]

# test_cases format
# test_cases = [
#     {
#         'prompt': '总会有地上的生灵，敢于直面雷霆的威光',
#     },
#     {
#         'prompt': '若你困于无风之地，我将为你奏响高天之歌',
#     },
#     {
#         'prompt': '倘若七神心中各有标尺，将神之眼赐予众生',
#     }
# ]

if use_wandb:
    wandb.config.update({"description": description,
                        "test_cases": test_cases})

In [9]:
generate_optimal_prompt(description, test_cases, NUMBER_OF_PROMPTS, use_wandb)

  3%|█                                 | 2/63 [00:04<02:09,  2.13s/it]

Draw


  5%|█▌                                | 3/63 [00:08<02:59,  3.00s/it]

Winner: As an AI specializing in psychology and intimate relationships with a focus on parenting issues, your job is to provide understanding and empathetic responses to a variety of questions about parenting. Your answers should be grounded in psychological best practices and empathy. Keep in mind that the person asking the question might be in a state of stress or confusion, so tailor your responses in a way that is supportive and non-judgmental. Use your understanding of human emotions and reactions to provide the most useful and comforting advice.


  6%|██▏                               | 4/63 [00:12<03:24,  3.47s/it]

...

Draw


100%|█████████████████████████████████| 63/63 [05:46<00:00,  4.13s/it]

Draw


100%|█████████████████████████████████| 63/63 [05:51<00:00,  5.57s/it]

Draw
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|                                                                                                                                                                                                                                                                                                                            